In [2]:
import cv2
import face_recognition
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class FaceLandmarksGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Real-time Facial Landmark Detection")
        self.root.configure(bg="#2c2c2c")
        self.root.geometry("1000x600+100+100")
        
        # Header with Project Name
        self.header_label = tk.Label(self.root, text="Real-time Facial Landmark Detection",
                                     font=("Helvetica", 20, "bold"), bg="#424242", fg="white")
        self.header_label.pack(fill="x", pady=10)

        # Sidebar for buttons
        sidebar_frame = tk.Frame(self.root, bg="#424242", width=200, height=600)
        sidebar_frame.pack(side="left", fill="y")

        # 'Menu' button to toggle options
        menu_button = tk.Button(sidebar_frame, text="Menu", command=self.toggle_menu, bg="#616161", fg="white",
                                font=("Helvetica", 12), relief="flat", bd=2)
        menu_button.pack(pady=10, padx=10, fill="x")

        # Container for menu options
        self.buttons_container = tk.Frame(sidebar_frame, bg="#424242")
        self.buttons_container.pack_forget()

        button_params = {
            "bg": "#757575", "fg": "white", "font": ("Helvetica", 12), "relief": "flat", "bd": 2
        }

        # Buttons inside "Menu"
        self.upload_button = tk.Button(self.buttons_container, text="Upload Image", command=self.upload_image,
                                       **button_params)
        self.upload_button.pack(pady=10, padx=10, fill="x")

        self.show_button = tk.Button(self.buttons_container, text="Show Landmarks", command=self.show_landmarks,
                                     **button_params)
        self.show_button.pack(pady=10, padx=10, fill="x")

        self.save_button = tk.Button(self.buttons_container, text="Save Image", command=self.save_image,
                                     **button_params)
        self.save_button.pack(pady=10, padx=10, fill="x")

        self.reset_button = tk.Button(self.buttons_container, text="Reset Image", command=self.reset_image,
                                      **button_params)
        self.reset_button.pack(pady=10, padx=10, fill="x")

        self.start_camera_button = tk.Button(self.buttons_container, text="Start Live Feed",
                                             command=self.start_live_feed, **button_params)
        self.start_camera_button.pack(pady=10, padx=10, fill="x")

        self.stop_camera_button = tk.Button(self.buttons_container, text="Close Live Feed",
                                            command=self.stop_live_feed, **button_params)
        self.stop_camera_button.pack(pady=10, padx=10, fill="x")

        # Image display label
        self.image_label = tk.Label(self.root, bg="#2c2c2c")
        self.image_label.pack(padx=10, pady=10, expand=True)

        # Footer with copyright and ID
        self.footer_label = tk.Label(self.root, text="© 2024 All rights reserved | ID: bc200403230",
                                     font=("Helvetica", 10), bg="#2c2c2c", fg="white")
        self.footer_label.pack(side="bottom", fill="x", pady=10)

        # Variables
        self.image_path = None
        self.displayed_image = None
        self.modified_image = None
        self.camera_running = False

    def toggle_menu(self):
        if self.buttons_container.winfo_ismapped():
            self.buttons_container.pack_forget()
        else:
            self.buttons_container.pack(padx=10, pady=10, fill="y")

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if file_path:
            self.image_path = file_path
            self.show_image()

    def show_image(self):
        if not self.image_path:
            messagebox.showerror("Error", "Please select an image first.")
            return
        image = Image.open(self.image_path)
        self.displayed_image = ImageTk.PhotoImage(image)
        self.image_label.configure(image=self.displayed_image)
        self.image_label.image = self.displayed_image
        self.modified_image = image

    def show_landmarks(self):
        if not self.image_path:
            messagebox.showerror("Error", "Please upload an image first.")
            return

        image = cv2.imread(self.image_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_image)

        if len(face_locations) > 0:
            face_landmarks = face_recognition.face_landmarks(rgb_image, face_locations)
            for face_landmark in face_landmarks:
                for facial_feature in face_landmark.keys():
                    for point in face_landmark[facial_feature]:
                        cv2.circle(image, point, 2, (0, 255, 0), -1)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(image_rgb)
            self.displayed_image = ImageTk.PhotoImage(img_pil)
            self.image_label.configure(image=self.displayed_image)
            self.image_label.image = self.displayed_image
            self.modified_image = img_pil
        else:
            messagebox.showerror("Error", "No faces found in the image.")

    def save_image(self):
        if self.modified_image is None:
            messagebox.showerror("Error", "No image to save.")
            return
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
        if save_path:
            self.modified_image.save(save_path)
            messagebox.showinfo("Success", "Image saved successfully.")

    def reset_image(self):
        if self.image_path:
            self.show_image()

    def start_live_feed(self):
        self.camera_running = True
        self.capture = cv2.VideoCapture(0)
        self.update_live_feed()

    def update_live_feed(self):
        if self.camera_running:
            ret, frame = self.capture.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb_frame)
                face_landmarks = face_recognition.face_landmarks(rgb_frame, face_locations)

                for landmarks in face_landmarks:
                    for points in landmarks.values():
                        for point in points:
                            cv2.circle(frame, point, 2, (0, 255, 0), -1)

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(rgb_frame)
                self.displayed_image = ImageTk.PhotoImage(img_pil)
                self.image_label.configure(image=self.displayed_image)
                self.image_label.image = self.displayed_image

            self.root.after(10, self.update_live_feed)

    def stop_live_feed(self):
        if self.camera_running:
            self.camera_running = False
            self.capture.release()
            self.image_label.configure(image="")
            messagebox.showinfo("Info", "Live feed closed.")

if __name__ == "__main__":
    root = tk.Tk()
    app = FaceLandmarksGUI(root)
    root.mainloop()
